In [1]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils import *
import vgg16
import pandas as pd
import numpy as np
from glob import glob
from keras.layers import Dense, Conv2D, BatchNormalization, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.metrics import log_loss
%matplotlib inline

data_dir = os.path.join(os.getcwd(), 'data', 'whale')
img_dir  = os.path.join(os.getcwd(), 'data', 'whale', 'imgs')
batch_size=64
num_class = 447
img_shape=(3, 224, 224)
target_size = (224,224)

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
data_dir

'/home/ubuntu/courses/deeplearning1/nbs/data/whale'

In [3]:
img_dir

'/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs'

In [22]:
ImageDataGenerator?

In [67]:
def get_tr_batches(dr = img_dir+ '/train', gen = ImageDataGenerator(), batch_size=batch_size):
    return gen.flow_from_directory(dr, shuffle=True, target_size=target_size, batch_size=64)

def get_val_batches(dr = img_dir+ '/valid', gen = ImageDataGenerator(), batch_size=batch_size):
    return gen.flow_from_directory(dr, shuffle=False, target_size=target_size, batch_size=batch_size)


def get_test_batches(dr = img_dir+ '/test', gen = ImageDataGenerator(), batch_size=batch_size):
    return gen.flow_from_directory(dr, shuffle=False, target_size=target_size, batch_size=batch_size)

tr_features = load_array('vgg_conv_tr.bc')
val_features = load_array('vgg_val_tr.bc')


tr_labels = load_array('whale_tr_classes.bc')
val_labels = load_array('whale_val_classes.bc')


In [5]:
def fit_model(model, epochs, tr_b = get_tr_batches(), val_b = get_val_batches()):
    model.fit_generator(tr_b, tr_b.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, 
                    nb_epoch=epochs)

Found 3635 images belonging to 447 classes.
Found 909 images belonging to 447 classes.


In [28]:
def get_model():
    vgg = Vgg16()
    model = vgg.model
    last_conv = [i for i, v in enumerate(model.layers) if type(v) is Conv2D][-1]
    for layer in model.layers[: last_conv+1]: layer.trainable=False
    model.layers = model.layers[:-1]
    model.add(Dense(num_class, activation='softmax'))
    return model

In [30]:
model = get_model()

In [31]:
augmenter = ImageDataGenerator(rotation_range=180, horizontal_flip=True, vertical_flip=True, zoom_range=0.10)

In [32]:
tr_batches = get_tr_batches(gen=augmenter)

Found 3635 images belonging to 447 classes.


In [34]:
val_b = get_val_batches()

Found 909 images belonging to 447 classes.


In [39]:
model.compile(Adam(0.001), 'categorical_crossentropy', metrics=['accuracy'])

In [37]:
model.fit_generator?

In [40]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=10)

Epoch 1/10
3635/3635 [==============================] - 122s - loss: 6.0966 - acc: 0.0019 - val_loss: 6.0862 - val_acc: 0.0044
Epoch 2/10
3635/3635 [==============================] - 122s - loss: 6.0752 - acc: 0.0044 - val_loss: 6.0723 - val_acc: 0.0044
Epoch 3/10
3635/3635 [==============================] - 122s - loss: 6.0570 - acc: 0.0061 - val_loss: 6.0599 - val_acc: 0.0044
Epoch 4/10
3635/3635 [==============================] - 122s - loss: 6.0406 - acc: 0.0061 - val_loss: 6.0487 - val_acc: 0.0044
Epoch 5/10
3635/3635 [==============================] - 122s - loss: 6.0258 - acc: 0.0052 - val_loss: 6.0393 - val_acc: 0.0066
Epoch 6/10
3635/3635 [==============================] - 122s - loss: 6.0126 - acc: 0.0102 - val_loss: 6.0308 - val_acc: 0.0099
Epoch 7/10
3635/3635 [==============================] - 122s - loss: 6.0006 - acc: 0.0105 - val_loss: 6.0232 - val_acc: 0.0099
Epoch 8/10
3635/3635 [==============================] - 122s - loss: 5.9896 - acc: 0.0105 - val_loss: 6.0168 - 

In [41]:
model.save_weights('augment_10_epochs.h5')

In [42]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=3)

Epoch 1/3
3635/3635 [==============================] - 122s - loss: 5.9625 - acc: 0.0096 - val_loss: 6.0011 - val_acc: 0.0099
Epoch 2/3
3635/3635 [==============================] - 123s - loss: 5.9550 - acc: 0.0105 - val_loss: 5.9972 - val_acc: 0.0099
Epoch 3/3
3635/3635 [==============================] - 122s - loss: 5.9471 - acc: 0.0113 - val_loss: 5.9938 - val_acc: 0.0099


In [43]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 122s - loss: 5.9418 - acc: 0.0083 - val_loss: 5.9907 - val_acc: 0.0099


In [44]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 122s - loss: 5.9328 - acc: 0.0118 - val_loss: 5.9885 - val_acc: 0.0099


In [45]:
model.save_weights('augment__5.9885.h5')

In [46]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 122s - loss: 5.9288 - acc: 0.0118 - val_loss: 5.9859 - val_acc: 0.0099


In [47]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 122s - loss: 5.9242 - acc: 0.0099 - val_loss: 5.9838 - val_acc: 0.0099


In [48]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 123s - loss: 5.9226 - acc: 0.0102 - val_loss: 5.9822 - val_acc: 0.0099


In [48]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 123s - loss: 5.9226 - acc: 0.0102 - val_loss: 5.9822 - val_acc: 0.0099


In [49]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 123s - loss: 5.9191 - acc: 0.0107 - val_loss: 5.9810 - val_acc: 0.0099


In [50]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 122s - loss: 5.9101 - acc: 0.0105 - val_loss: 5.9797 - val_acc: 0.0099


In [51]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 123s - loss: 5.9196 - acc: 0.0105 - val_loss: 5.9789 - val_acc: 0.0099


In [52]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 123s - loss: 5.9007 - acc: 0.0102 - val_loss: 5.9786 - val_acc: 0.0099


In [53]:
model.save_weights('vgg_augment_5.9786.h5')

In [54]:
model.optimizer.lr = 0.0001

In [55]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 122s - loss: 5.9069 - acc: 0.0107 - val_loss: 5.9778 - val_acc: 0.0099


In [56]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 123s - loss: 5.9052 - acc: 0.0113 - val_loss: 5.9771 - val_acc: 0.0099


In [57]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 123s - loss: 5.8968 - acc: 0.0102 - val_loss: 5.9769 - val_acc: 0.0099


In [58]:
model.save_weights('vgg_aug_5.9769_.h5')

In [59]:
model.optimizer.lr = 0.00001

In [60]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 122s - loss: 5.9007 - acc: 0.0110 - val_loss: 5.9773 - val_acc: 0.0099


In [61]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 123s - loss: 5.8869 - acc: 0.0116 - val_loss: 5.9768 - val_acc: 0.0099


In [62]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 123s - loss: 5.8931 - acc: 0.0099 - val_loss: 5.9767 - val_acc: 0.0099


In [63]:
model.optimizer.lr = 0.000001

In [64]:
model.fit_generator(tr_batches, 
                    tr_batches.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, nb_epoch=1)

Epoch 1/1
3635/3635 [==============================] - 122s - loss: 5.8885 - acc: 0.0113 - val_loss: 5.9768 - val_acc: 0.0099


In [68]:
test_b = get_test_batches(batch_size=16)

Found 6925 images belonging to 1 classes.


In [70]:
preds = model.predict_generator(test_b, test_b.nb_sample)

In [74]:
from create_whale_sub import create_sub

In [71]:
preds.shape

(6925, 447)

In [75]:
sub = create_sub(preds)

Found 3635 images belonging to 447 classes.
Found 6925 images belonging to 1 classes.


In [77]:
sub.to_csv('vgg_aug.csv', index=False)

In [78]:
from IPython.display import FileLink
FileLink('vgg_aug.csv')

/home/ubuntu/courses/deeplearning1/nbs/vgg_aug.csv